In [275]:
import os
import sys

In [276]:
import pandas as pd
from pandas import Series, DataFrame

In [277]:
!ls ../Data/Movie_100k

README       u.genre      u.user       u2.test      u4.test      ua.test
allbut.pl    u.info       u1.base      u3.base      u5.base      ub.base
mku.sh       u.item       u1.test      u3.test      u5.test      ub.test
u.data       u.occupation u2.base      u4.base      ua.base


In [278]:
all_ratings = pd.read_csv('../Data/Movie_100k/u.data', delimiter="\t", header=None, names = ["UserID", "MovieID", "Rating"
                                                                                 , "Datetime"])

#data_folder = os.path.join (os.path.expanduser("~"), "Data", "download","ml-100k")
#ratings_filename = os.path.join(data_folder,"u.data")

In [279]:
#all_ratings.rename(columns={'userId': 'UserID', 
#                                        'movieId': 'MovieID',
#                                        'rating': 'Rating',
#                                        'timestamp': 'Datetime'}, inplace=True)


#all_ratings = pd.read_csv(ratings_filename, delimiter="\t", header=None, names = ["UserID", "MovieID", "Rating"
#                                                                                 , "Datetime"])

In [280]:
all_ratings[:5]

,UserID,MovieID,Rating,Datetime
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [281]:
#解析时间戳数据
all_ratings["Datetime"] = pd.to_datetime(all_ratings['Datetime'],
                                        unit = 's')

In [282]:
#运行前五行
all_ratings[:5]

,UserID,MovieID,Rating,Datetime
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16


In [283]:
#用户喜欢某部电影
all_ratings["Favorable"] = all_ratings["Rating"] > 3

In [284]:
all_ratings[10:15]

,UserID,MovieID,Rating,Datetime,Favorable
10,62,257,2,1997-11-12 22:07:14,False
11,286,1014,5,1997-11-17 15:38:45,True
12,200,222,5,1997-10-05 09:05:40,True
13,210,40,3,1998-03-27 21:59:54,False
14,224,29,3,1998-02-21 23:40:57,False


In [287]:
#训练集，取前200名用户
ratings = all_ratings[all_ratings['UserID'].isin(range(200))]

In [288]:
favorable_ratings = ratings[ratings["Favorable"]]

In [289]:
favorable_ratings[:20]

,UserID,MovieID,Rating,Datetime,Favorable
16,122,387,5,1997-11-11 17:47:39,True
20,119,392,4,1998-01-30 16:13:34,True
21,167,486,4,1998-04-16 14:54:12,True
26,38,95,5,1998-04-13 01:14:54,True
28,63,277,4,1997-10-01 23:10:01,True
29,160,234,5,1997-10-14 20:33:05,True
35,157,274,4,1998-02-07 22:33:55,True
39,7,32,4,1998-03-31 13:28:52,True
40,10,16,4,1997-10-26 18:01:17,True
48,99,4,5,1998-02-03 15:18:17,True


In [290]:
# v.values储存为frozenset，便于快速判断用户是否为某部电影打过分
# 建立字典，uid为key，movieid为value

favorable_reviews_by_users = dict((k, frozenset(v.values))
                                 for k, v in favorable_ratings.groupby("UserID")["MovieID"])

In [292]:
favorable_reviews_by_users

{1: frozenset({1,
            3,
            6,
            7,
            9,
            12,
            13,
            14,
            15,
            16,
            18,
            19,
            20,
            22,
            23,
            25,
            28,
            32,
            33,
            39,
            42,
            43,
            44,
            45,
            46,
            47,
            48,
            50,
            51,
            52,
            55,
            56,
            57,
            58,
            59,
            60,
            61,
            64,
            65,
            66,
            68,
            72,
            75,
            76,
            77,
            79,
            80,
            81,
            82,
            84,
            86,
            87,
            88,
            89,
            90,
            91,
            93,
            95,
            96,
            98,
            100,
            106,
        

In [293]:
#创建数据框，了解每部电影影迷数量
num_favorable_by_movie = ratings[["MovieID", "Favorable"]].groupby("MovieID").sum()

In [294]:
#每部电影有过的好评量

num_favorable_by_movie[:20]

,Favorable
MovieID,
1,66.0
2,5.0
3,4.0
4,21.0
5,6.0
6,6.0
7,67.0
8,36.0
9,53.0


In [295]:
#最受欢迎的五部电影
num_favorable_by_movie.sort_values("Favorable", ascending=False)[:5]

,Favorable
MovieID,
50,100.0
100,89.0
258,83.0
181,79.0
174,74.0


# Apriori算法

In [296]:
#初始化一个字典
frequent_itemsets = {}

In [243]:
#设置最小支持度
#电影得在数据集中出现50次

min_support = 50

In [301]:
#Apriori算法第一步，为每一部电影生成只包含它自己的项集，检测它是否够频繁 --- 出现次数大于50
#电影编号使用frozenset，后面要用到集合操作，他们也可用作字典的建（普通的集合不可以）

#dict(key,value)

frequent_itemsets[1] = dict((frozenset((movie_id,)),row["Favorable"])
                            for movie_id, row in num_favorable_by_movie.iterrows() #df.iterrows()--对表格进行遍历
                            if row["Favorable"] > min_support
                           )

In [302]:
frequent_itemsets[1]

{frozenset({1}): 66.0,
 frozenset({7}): 67.0,
 frozenset({9}): 53.0,
 frozenset({50}): 100.0,
 frozenset({56}): 67.0,
 frozenset({64}): 58.0,
 frozenset({79}): 58.0,
 frozenset({98}): 70.0,
 frozenset({100}): 89.0,
 frozenset({127}): 70.0,
 frozenset({172}): 59.0,
 frozenset({174}): 74.0,
 frozenset({181}): 79.0,
 frozenset({258}): 83.0,
 frozenset({286}): 59.0,
 frozenset({313}): 60.0}

In [245]:
#用一个函数来实现步骤2，3，它接收新发现的频繁项集，创建超集，检测频繁程度
#函数声明及字典初始化：

from collections import defaultdict
def find_frequent_itemsets(favorable_reviews_by_users, k_1_itemsets,
                          min_support):
    counts = defaultdict(int)
    for user, reviews in favorable_reviews_by_users.items():  #遍历所有用户和他们的打分数据
        
        #遍历前面找出的项集，判断它们是否是当前评分项集的子集
        for itemset in k_1_itemsets:
            
            if itemset.issubset(reviews):
                
                #遍历用户打过分却没有出现在项集里的电影，用它生成超集，更新该项集的计数
                for other_reviewed_movie in reviews - itemset:
                    current_superset = itemset | frozenset((other_reviewed_movie,))
                    counts[current_superset] += 1
                    
    
    #函数最后检测达到支持度要求的项集，看它的频繁程度够不够，并返回其中的频繁项集
    return dict([(itemset, frequency) for itemset, frequency in counts.items() if frequency >= min_support])

In [246]:
#创建循环，运行Aprior算法，存储算法运行过程中发现的新项集
#k表示即将发现的频繁项集的长度
#键k-1从frequet_itemsets字典中获取刚发现的频繁项集
#新发现的频繁项集以长度为键，将其保存到字典中

for k in range(2, 20):
    cur_frequent_itemsets = find_frequent_itemsets(favorable_reviews_by_users,
                              frequent_itemsets[k-1],
                                  min_support)
    frequent_itemsets[k] = cur_frequent_itemsets
        
    #如果在上述循环中没有找到任何新的频繁项集，就跳出循环
    if len(cur_frequent_itemsets) == 0:
        print("Did not find any frequent itemsets of length {}".
             format(k))
        sys.stdout.flush()
        break
    
    #如果确实找到频繁项集，我们也让程序输出信息，告知我们会再次运行
    else:
        print("I found {} frequent itemsets of length {}".format(len(cur_frequent_itemsets),k))
        sys.stdout.flush()
        

#删除长度为1的项集

del frequent_itemsets[1]

I found 93 frequent itemsets of length 2
I found 295 frequent itemsets of length 3
I found 593 frequent itemsets of length 4
I found 785 frequent itemsets of length 5
I found 677 frequent itemsets of length 6
I found 373 frequent itemsets of length 7
I found 126 frequent itemsets of length 8
I found 24 frequent itemsets of length 9
I found 2 frequent itemsets of length 10
Did not find any frequent itemsets of length 11


# 抽取关联规则

In [247]:
# Apriori算法结束后，我们得到了一系列频繁项集，这还不算是真正意义上的关联规则。
#频繁项集是一族达到最小支持度的项目，而关联规则由前提和结论组成
#我们可以从频繁项集中抽取出关联规则， 把其中几部电影作为基本前提，另几部而电影作为结论组成如下规则：
#如果用户喜欢前提中的所有电影，那么他们也会喜欢结论中的电影


#通过遍历不同长度的频繁项集，为每个项集生成规则

        #遍历项集中每部电影，把它作为结论
        #项集中的其他电影作为前提，用前提和结论丞备选规则

#如此就得到大量备选规则
#查看前五条规则



candidate_rules = []
for itemset_length, itemset_counts in frequent_itemsets.items():
    for itemset in itemset_counts.keys():
        
        for conclusion in itemset:
            premise = itemset - set((conclusion,))
            candidate_rules.append((premise, conclusion))

print(candidate_rules[:5])
    

[(frozenset({7}), 1), (frozenset({1}), 7), (frozenset({50}), 1), (frozenset({1}), 50), (frozenset({1}), 56)]


In [248]:
#计算每条规则置信度

#先创建两个字典，用来存储规则应验（正例）和规则不适用（反例）的次数

    #遍历所有用户及其喜欢的电影数据，在这个过程中遍历每条关联规则
    
    #测试每天规则的前提对用户是否使用，即用户是否喜欢前提中的所有电影
    #如果前提符合，看用户是否喜欢结论中的电影，是的话则规则试用，反之则规则不适用

correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)

for user, reviews in favorable_reviews_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1

In [249]:
#用规则应验的次数除以前提条件出现的总次数，计算每天规则的置信度

rule_confidence = {candidate_rule: correct_counts[candidate_rule]
                  / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
                      for candidate_rule in candidate_rules}

In [250]:
candidate_rule

(frozenset({7, 50, 64, 79, 98, 100, 172, 174, 181}), 56)

In [251]:
#rule_confidence.items()

In [262]:
#对置信度字典进行排序后，输出置信度最高的前五条规则

from operator import itemgetter
sorted_confidence = sorted(rule_confidence.items(),key=itemgetter(1), reverse=True)

for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise, conclusion))
    print(" - Confidence : {0:.3f}".format(rule_confidence[(premise, conclusion)]))
    print("")

Rule #1
Rule: If a person recommends frozenset({98, 181}) they will also recommend 50
 - Confidence : 1.000

Rule #2
Rule: If a person recommends frozenset({172, 79}) they will also recommend 174
 - Confidence : 1.000

Rule #3
Rule: If a person recommends frozenset({258, 172}) they will also recommend 174
 - Confidence : 1.000

Rule #4
Rule: If a person recommends frozenset({1, 181, 7}) they will also recommend 50
 - Confidence : 1.000

Rule #5
Rule: If a person recommends frozenset({1, 172, 7}) they will also recommend 174
 - Confidence : 1.000



In [263]:
#添加电影名字

!ls ../Data/movie_100k

README       u.genre      u.user       u2.test      u4.test      ua.test
allbut.pl    u.info       u1.base      u3.base      u5.base      ub.base
mku.sh       u.item       u1.test      u3.test      u5.test      ub.test
u.data       u.occupation u2.base      u4.base      ua.base


In [264]:
movie_name_data = pd.read_csv('../Data/Movie_100k/u.item', delimiter="|", header=None, encoding = "mac-roman")

movie_name_data.columns = ["MovieID", "Title", "Release Date",
                          "Video Release", "IMDB", "<UNK>", "Action", "Adventure",
                          "Animation", "Children's", "Comedy", "Crime", "Documentary",
                          "Drama", "Fantasy", "Film-Noir",
                          "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller",
                          "War", "Western"]

In [265]:
movie_name_data

,MovieID,Title,Release Date,Video Release,IMDB,<UNK>,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,8,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995),0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8,9,Dead Man Walking (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Dead%20Man%20...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,Richard III (1995),22-Jan-1996,NaN,http://us.imdb.com/M/title-exact?Richard%20III...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [266]:
#创建一个用电影编号获取名称的函数
#用title_object的value属性获取电影名称（不是series对象）

def get_movie_name(movie_id):
    title_object = movie_name_data[movie_name_data["MovieID"] == movie_id]["Title"]
    title = title_object.values[0]
    return title


In [268]:
#调整之前的代码，这样就能在输出的规则中显示电影名称

for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx) for idx
                             in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
    print(" - Confidence : {0:.3f}".format(rule_confidence[(premise, conclusion)]))
print("")
    

Rule #1
Rule: If a person recommends Silence of the Lambs, The (1991), Return of the Jedi (1983) they will also recommend Star Wars (1977)
 - Confidence : 1.000
Rule #2
Rule: If a person recommends Empire Strikes Back, The (1980), Fugitive, The (1993) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence : 1.000
Rule #3
Rule: If a person recommends Contact (1997), Empire Strikes Back, The (1980) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence : 1.000
Rule #4
Rule: If a person recommends Toy Story (1995), Return of the Jedi (1983), Twelve Monkeys (1995) they will also recommend Star Wars (1977)
 - Confidence : 1.000
Rule #5
Rule: If a person recommends Toy Story (1995), Empire Strikes Back, The (1980), Twelve Monkeys (1995) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence : 1.000



# 评估

In [270]:
#看每天规则的表现，寻找好的规则
#抽取没有用于训练的数据建立测试集，训练集用前200名用户打分的数据，测试集用剩下的数据。

test_dataset = all_ratings[~all_ratings['UserID'].isin(range(200))]
test_favorable = test_dataset[test_dataset["Favorable"]]
test_favorable_by_users = dict((k, frozenset(v.values)) for k, v
                              in test_favorable.groupby("UserID")["MovieID"])

In [272]:
#用测试数据计算规则应验的数量

correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)

for user, reviews in test_favorable_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1
                
                

In [273]:
#计算置信度

test_confidence = {candidate_rule: correct_counts[candidate_rule]
                  / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
                      for candidate_rule in candidate_rules}

In [274]:
#用输出电影名称而不是电影编号表示的最佳关联规则

for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx) for idx
                             in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
    print(" - Train Confidence: {0:.3f}".format(rule_confidence.get((premise, conclusion), -1)))
    print(" - Test Confidence: {0:.3f}".format(test_confidence.get((premise, conclusion), -1)))
print("")


Rule #1
Rule: If a person recommends Silence of the Lambs, The (1991), Return of the Jedi (1983) they will also recommend Star Wars (1977)
 - Train Confidence: 1.000
 - Test Confidence: 0.936
Rule #2
Rule: If a person recommends Empire Strikes Back, The (1980), Fugitive, The (1993) they will also recommend Raiders of the Lost Ark (1981)
 - Train Confidence: 1.000
 - Test Confidence: 0.876
Rule #3
Rule: If a person recommends Contact (1997), Empire Strikes Back, The (1980) they will also recommend Raiders of the Lost Ark (1981)
 - Train Confidence: 1.000
 - Test Confidence: 0.841
Rule #4
Rule: If a person recommends Toy Story (1995), Return of the Jedi (1983), Twelve Monkeys (1995) they will also recommend Star Wars (1977)
 - Train Confidence: 1.000
 - Test Confidence: 0.932
Rule #5
Rule: If a person recommends Toy Story (1995), Empire Strikes Back, The (1980), Twelve Monkeys (1995) they will also recommend Raiders of the Lost Ark (1981)
 - Train Confidence: 1.000
 - Test Confidence: 0.